In [201]:
import numpy as np
import pandas as pd
import yfinance as yf
import pandas_datareader as pdr
import seaborn as sns
from matplotlib import pyplot as plt
from scipy.stats import norm
import datetime

In [203]:
def black_scholes(S, K, T, r, sigma, option_type='call'):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    if option_type == 'call':
        option_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == 'put':
        option_price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    else:
        raise ValueError("Invalid option type. Use 'call' or 'put'.")
    
    return option_price

In [215]:
def Price_now(ticker,strike,option_type,maturity_period=0,look_back_period=30):
    Start_date = datetime.date.today() - datetime.timedelta(days=2*look_back_period) # prepare enough data for rolling
    df_Stock_price = yf.download(ticker,start = Start_date)
    symbol = yf.Ticker(ticker)

    for col in df_Stock_price.columns:
        df_Stock_price[f"lr {col}"] = np.log(df_Stock_price[f'{col}'] / df_Stock_price[f'{col}'].shift(1))
        df_Stock_price[f"vol {col}"] = df_Stock_price[f"lr {col}"].rolling(window=look_back_period).std()
    
    df_risk_free_rate = pdr.DataReader("DGS3MO", 'fred', Start_date).dropna()
    df_risk_free_rate["DGS3MO"] = df_risk_free_rate["DGS3MO"]/100
    risk_free_rate = df_risk_free_rate["DGS3MO"].iloc[-1]
    
    Current_price = symbol.info['currentPrice']
    expiration = symbol.options[maturity_period]
    option_chain = symbol.option_chain(expiration)
    T_expiration = ((pd.Timestamp(expiration) - pd.Timestamp(datetime.date.today())).days)/360
    sigma = np.sqrt(360)*np.sqrt((df_Stock_price["vol Open"]**2 + df_Stock_price["vol Close"]**2 + df_Stock_price["vol High"]**2 + df_Stock_price["vol Low"]**2)/4).iloc[-1]

    Option_price = black_scholes(Current_price,strike,T_expiration,risk_free_rate,sigma,option_type)
    print(f'Price for {ticker} {option_type} at {strike} is {Option_price:.2f} USD')
    return Option_price

In [217]:
P=Price_now('NVDA',136,'call')

[*********************100%%**********************]  1 of 1 completed


Price for NVDA call at 136 is 0.91 USD
